In [271]:
import tensorflow as tf
from tensorflow import keras
from keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import precision_score, recall_score
from sklearn.metrics import confusion_matrix, classification_report
from keras.optimizers.legacy import Adam
import os

In [272]:
train_data_dir = "./data/train"
test_data_dir = "./data/test"

image_size = (224, 224)
batch_size = 32

train_datagen = ImageDataGenerator(
    rescale=1.0 / 255.0,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    zoom_range=0.1,
    validation_split=0.2
)

In [273]:
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='training',
    shuffle=True
)

Found 168 images belonging to 2 classes.


In [274]:
validation_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation',
    shuffle=True
)

Found 41 images belonging to 2 classes.


In [289]:
def calculate_metrics(y_true, y_pred):
    precision = precision_score(y_true, y_pred, average='weighted')
    recall = recall_score(y_true, y_pred, average='weighted')
    return precision, recall


model = keras.Sequential([
    keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    keras.layers.MaxPooling2D(2, 2),
    keras.layers.Conv2D(32, (3, 3), activation='relu'),
    keras.layers.MaxPooling2D(2, 2),
    keras.layers.Flatten(),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(2, activation='softmax')
])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [290]:
train_losses = []
train_accuracies = []
val_losses = []
val_accuracies = []

class MetricsCallback(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        print(
            f'Epoch {epoch + 1} | Train Loss: {logs["loss"]:.4f}  - Train Accuracy: {logs["accuracy"]:.4f} | Validation Loss: {logs["val_loss"]:.4f} - Validation Accuracy: {logs["val_accuracy"]:.4f} ')
        train_losses.append(logs['loss'])
        train_accuracies.append(logs['accuracy'])
        val_losses.append(logs['val_loss'])
        val_accuracies.append(logs['val_accuracy'])

metrics_callback = MetricsCallback()


In [291]:
history= model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=10,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size,
    callbacks=[metrics_callback]
)

Epoch 1/10
5/5 [==============================] - 4s 649ms/step - loss: 0.4446 - accuracy: 0.8000 - val_loss: 0.1159 - val_accuracy: 1.0000
Epoch 2/10
5/5 [==============================] - 2s 485ms/step - loss: 0.0680 - accuracy: 1.0000 - val_loss: 0.0182 - val_accuracy: 1.0000
Epoch 3/10
5/5 [==============================] - 3s 550ms/step - loss: 0.0069 - accuracy: 1.0000 - val_loss: 0.0015 - val_accuracy: 1.0000
Epoch 4/10
5/5 [==============================] - 2s 448ms/step - loss: 0.0049 - accuracy: 1.0000 - val_loss: 4.3288e-04 - val_accuracy: 1.0000
Epoch 5/10
5/5 [==============================] - 2s 447ms/step - loss: 7.7246e-04 - accuracy: 1.0000 - val_loss: 2.9333e-04 - val_accuracy: 1.0000
Epoch 6/10
5/5 [==============================] - 3s 518ms/step - loss: 0.0019 - accuracy: 1.0000 - val_loss: 3.3019e-04 - val_accuracy: 1.0000
Epoch 7/10
5/5 [==============================] - 2s 485ms/step - loss: 2.6721e-04 - accuracy: 1.0000 - val_loss: 2.2843e-04 - val_accuracy: 1.0

In [292]:
avg_train_loss = sum(train_losses) / len(train_losses)
avg_train_accuracy = sum(train_accuracies) / len(train_accuracies)
avg_val_loss = sum(val_losses) / len(val_losses)
avg_val_accuracy = sum(val_accuracies) / len(val_accuracies)

print(f'Average Training Loss: {avg_train_loss:.4f}')
print(f'Average Training Accuracy: {avg_train_accuracy:.4f}')
print(f'Average Validation Loss: {avg_val_loss:.4f}')
print(f'Average Validation Accuracy: {avg_val_accuracy:.4f}')

Average Training Loss: 0.0528
Average Training Accuracy: 0.9800
Average Validation Loss: 0.0138
Average Validation Accuracy: 1.0000


In [293]:
model.save("drowsiness_detection_model.h5")

/Users/mohith/anaconda3/lib/python3.11/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [294]:
test_datagen = ImageDataGenerator(rescale=1.0 / 255.0)
test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical'
)

Found 133 images belonging to 2 classes.


In [295]:
from sklearn.metrics import log_loss
true_labels = test_generator.classes
predictions = model.predict(test_generator)

for i, (prediction, true_label) in enumerate(zip(predictions, true_labels)):
    confidence_drowsy = prediction[0]
    confidence_not_drowsy = prediction[1]
    predicted_label = "DROWSY" if confidence_drowsy >= confidence_not_drowsy else "NOT DROWSY"
    label = "DROWSY" if true_label == 0 else "NOT DROWSY"
    print(
        f"Image {i + 1} - True Label: {label}, Predicted Label: {predicted_label},  Confidence Drowsy: {confidence_drowsy:.2f}, Confidence Not Drowsy: {confidence_not_drowsy:.2f}")

predicted_labels = predictions.argmax(axis=-1)
correct_predictions = (predicted_labels == true_labels)
accuracy = correct_predictions.sum() / len(true_labels)


test_classification_report = classification_report(true_labels, predicted_labels)
test_confusion_matrix = confusion_matrix(true_labels, predicted_labels)

true_positives = test_confusion_matrix[1, 1]  # The count of true positives
false_positives = test_confusion_matrix[0, 1]  # The count of false positives
false_negatives = test_confusion_matrix[1, 0]  # The count of false negatives

precision = true_positives / (true_positives + false_positives)
recall = true_positives / (true_positives + false_negatives)


print("\nTest Confusion Matrix:")
print(test_confusion_matrix)

print("\nTest Classification Report:")
print(test_classification_report)

# Print the accuracy
print(f"Test Accuracy: {accuracy:.4f}")
test_loss = log_loss(true_labels, predictions)
print(f"Test Loss: {test_loss:.4f}")

print("Test Precision:", precision)
print("Test Recall:", recall)


5/5 [==============================] - 1s 86ms/step
Image 1 - True Label: DROWSY, Predicted Label: NOT DROWSY,  Confidence Drowsy: 0.00, Confidence Not Drowsy: 1.00
Image 2 - True Label: DROWSY, Predicted Label: NOT DROWSY,  Confidence Drowsy: 0.00, Confidence Not Drowsy: 1.00
Image 3 - True Label: DROWSY, Predicted Label: NOT DROWSY,  Confidence Drowsy: 0.00, Confidence Not Drowsy: 1.00
Image 4 - True Label: DROWSY, Predicted Label: NOT DROWSY,  Confidence Drowsy: 0.01, Confidence Not Drowsy: 0.99
Image 5 - True Label: DROWSY, Predicted Label: NOT DROWSY,  Confidence Drowsy: 0.00, Confidence Not Drowsy: 1.00
Image 6 - True Label: DROWSY, Predicted Label: NOT DROWSY,  Confidence Drowsy: 0.00, Confidence Not Drowsy: 1.00
Image 7 - True Label: DROWSY, Predicted Label: NOT DROWSY,  Confidence Drowsy: 0.00, Confidence Not Drowsy: 1.00
Image 8 - True Label: DROWSY, Predicted Label: NOT DROWSY,  Confidence Drowsy: 0.00, Confidence Not Drowsy: 1.00
Image 9 - True Label: DROWSY, Predicted Labe

/Users/mohith/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/mohith/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/mohith/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
